<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fbigdatamagazine.es%2Fwp-content%2Fuploads%2F2023%2F02%2FFOTO-OK-BDM-DIG-DATA-MAGAZINE.jpg&f=1&nofb=1&ipt=4061921fa0da07483f83edb036d31f25545b2cae889c7eeefebd576f6e0fe5f4" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

En este tutorial, aprenderás cómo usar la **validación cruzada** para obtener mejores medidas del rendimiento del modelo.

# Introducción

El aprendizaje automático es un proceso iterativo.

Tendrás que tomar decisiones sobre qué variables predictoras utilizar, qué tipos de modelos emplear, qué argumentos proporcionar a esos modelos, etc. Hasta ahora, has tomado estas decisiones de manera guiada por los datos al medir la calidad del modelo con un conjunto de validación (o conjunto de retención).

Pero este enfoque tiene algunas desventajas. Para entenderlo, imagina que tienes un conjunto de datos con 5000 filas. Normalmente, conservarías alrededor del 20% de los datos como conjunto de validación, es decir, unas 1000 filas. Pero esto deja cierto grado de aleatoriedad en la determinación de las puntuaciones del modelo. Es decir, un modelo podría tener buen rendimiento sobre un conjunto específico de 1000 filas, aunque sería inexacto sobre un conjunto diferente de 1000 filas.

En un caso extremo, podrías imaginar que solo tienes 1 fila de datos en el conjunto de validación. Si comparas modelos alternativos, cuál de ellos hace la mejor predicción sobre un único punto de datos será, en gran medida, cuestión de suerte.

En general, cuanto mayor es el conjunto de validación, menor es la aleatoriedad (también conocida como "ruido") en nuestra medida de calidad del modelo, y más confiable será. Desafortunadamente, solo podemos obtener un conjunto de validación grande al eliminar filas de nuestro conjunto de entrenamiento, ¡y conjuntos de entrenamiento más pequeños significan peores modelos!



# ¿Qué es la validación cruzada?

En la **validación cruzada**, ejecutamos nuestro proceso de modelado sobre diferentes subconjuntos de los datos para obtener múltiples medidas de la calidad del modelo.

Por ejemplo, podríamos comenzar dividiendo los datos en 5 partes, cada una con el 20% del conjunto de datos completo. En este caso, decimos que hemos dividido los datos en 5 "**folds**" (pliegues).

![tut5_crossval](https://storage.googleapis.com/kaggle-media/learn/images/9k60cVA.png)

Luego, ejecutamos un experimento por cada fold:
- En el **Experimento 1**, usamos el primer fold como conjunto de validación (o retención) y todo lo demás como datos de entrenamiento. Esto nos da una medida de la calidad del modelo basada en un conjunto de retención del 20%.
- En el **Experimento 2**, dejamos fuera los datos del segundo fold (y usamos todo excepto el segundo fold para entrenar el modelo). El conjunto de retención se usa entonces para obtener una segunda estimación de la calidad del modelo.
- Repetimos este proceso, usando cada fold una vez como conjunto de retención. Al juntar todo esto, el 100% de los datos se utiliza como conjunto de retención en algún momento, y terminamos con una medida de la calidad del modelo basada en todas las filas del conjunto de datos (aunque no usemos todas las filas simultáneamente).

# ¿Cuándo deberías usar validación cruzada?

La validación cruzada proporciona una medida más precisa de la calidad del modelo, lo cual es especialmente importante si estás tomando muchas decisiones de modelado. Sin embargo, puede tardar más en ejecutarse, ya que estima múltiples modelos (uno por cada fold).

Entonces, dados estos compromisos, ¿cuándo deberías usar cada enfoque?
- _Para conjuntos de datos pequeños_, donde la carga computacional adicional no es un gran problema, deberías usar validación cruzada.
- _Para conjuntos de datos grandes_, un único conjunto de validación es suficiente. Tu código se ejecutará más rápido y podrías tener suficientes datos como para que no sea necesario reutilizar parte de ellos como retención.

No existe un umbral simple que defina qué constituye un conjunto de datos grande o pequeño. Pero si tu modelo tarda un par de minutos o menos en ejecutarse, probablemente valga la pena cambiar a validación cruzada.

Alternativamente, puedes ejecutar validación cruzada y ver si las puntuaciones de cada experimento son similares. Si cada experimento arroja resultados similares, un solo conjunto de validación probablemente sea suficiente.


# Ejemplo

Trabajaremos con los mismos datos que en el tutorial anterior. Cargamos los datos de entrada en `X` y los datos de salida en `y`.

In [1]:
import pandas as pd

# Leer los datos
data = pd.read_csv('/content/melb_data.csv')

# Seleccionar un subconjunto de predictores
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Seleccionar la variable objetivo
y = data.Price

Luego, definimos un pipeline que utiliza un imputador para completar los valores faltantes y un modelo de bosque aleatorio para hacer predicciones.

Aunque es _posible_ hacer validación cruzada sin pipelines, ¡es bastante complicado! Usar un pipeline hará que el código sea notablemente más sencillo.


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Definir un pipeline con un imputador y un modelo de bosque aleatorio
my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])


Obtenemos las puntuaciones de validación cruzada con la función [`cross_val_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) de scikit-learn. Establecemos el número de folds con el parámetro `cv`.


In [3]:
from sklearn.model_selection import cross_val_score

# Multiplicar por -1 ya que sklearn calcula el MAE *negativo*
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("Puntuaciones de MAE:\n", scores)


Puntuaciones de MAE:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


El parámetro `scoring` selecciona una medida de calidad del modelo que se va a reportar: en este caso, elegimos el error absoluto medio negativo (MAE). La documentación de scikit-learn muestra una [lista de opciones](http://scikit-learn.org/stable/modules/model_evaluation.html).

Es un poco sorprendente que especifiquemos el MAE *negativo*. Scikit-learn sigue una convención en la que todas las métricas están definidas de forma que un valor alto sea mejor. Usar valores negativos aquí les permite ser consistentes con esa convención, aunque el MAE negativo es casi inexistente fuera de este contexto.

Normalmente queremos una única medida de calidad del modelo para comparar modelos alternativos. Por eso tomamos el promedio entre los experimentos.


In [4]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
277707.3795913405


# Conclusión

Usar validación cruzada proporciona una medida mucho más precisa de la calidad del modelo, con el beneficio adicional de simplificar nuestro código: observa que ya no necesitamos llevar un control separado de los conjuntos de entrenamiento y validación. Por lo tanto, especialmente en conjuntos de datos pequeños, ¡es una muy buena mejora!
